# EHR A3
## GROUP: B
## MEMBERS: ERNEST CEBALLOS, JÚLIA GALIMANY, ORIOL GALIMANY


In [1]:
library(dplyr)
library(tidyr)
library(tibble)
library(lubridate)
library(readr)
library(stringr)
library(ggplot2)
library(data.table)
library(odbc)
library(RMariaDB)
library(DBI)


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union



Attaching package: ‘lubridate’


The following objects are masked from ‘package:base’:

    date, intersect, setdiff, union



Attaching package: ‘data.table’


The following objects are masked from ‘package:lubridate’:

    hour, isoweek, mday, minute, month, quarter, second, wday, week,
    yday, year


The following objects are masked from ‘package:dplyr’:

    between, first, last




In [2]:
con <- dbConnect(
  drv = RMariaDB::MariaDB(),
  username = "ernest.ceballos",
  password = "EeWot5Bu",
  host = "ehr3.deim.urv.cat",
  dbname = "mimiciiiv14",
  port = 3306
)

# PART 1: COHORT 

### Cohort definition
We performed a retrospective observational study using the Medical Information Mart for Intensive Care III (MIMIC-III), version 1.4, a publicly available database containing de-identified health data from over 40,000 adult and neonatal ICU admissions at Beth Israel Deaconess Medical Center between 2001 and 2012 [1]. Our cohort comprised all adult patients aged 16 years or older with at least one ICU admission recorded in the ICUSTAYS table. Age was calculated as the difference in years between the ICU admission time (INTIME) and the patient’s date of birth. For patients with multiple ICU stays, each stay was considered an independent observation. Neonatal and pediatric admissions (age < 16 years) were excluded. The final analytical cohort included all qualifying ICU admissions available in the database, without further exclusion criteria.


In [3]:
cohort <- tbl(con, "ICUSTAYS") %>%
  # Unir amb admissions per obtenir hospital_expire_flag i admittime
  inner_join(tbl(con, "ADMISSIONS"), by = c("HADM_ID" = "HADM_ID", "SUBJECT_ID" = "SUBJECT_ID")) %>%
  # Unir amb patients per obtenir data de naixement i gènere
  inner_join(tbl(con, "PATIENTS"), by = c("SUBJECT_ID" = "SUBJECT_ID")) %>%
  # Calcular edat en anys
  mutate(
    age = year(INTIME) - year(DOB)
  ) %>%
  # Filtrar només adults (>=16 anys)
  filter(age >= 16) %>%
  # Seleccionar variables rellevants
  select(
    HADM_ID,
    SUBJECT_ID,
    ICUSTAY_ID,
    age,
    GENDER,
    INTIME,
    OUTTIME,
    HOSPITAL_EXPIRE_FLAG
  ) %>%
  distinct() %>%
  # Portar dades a memòria local
  collect()

# Renombrar columnes a minúscules per a facilitat
colnames(cohort) <- tolower(colnames(cohort))

# Verificació bàsica de la cohort
cat("Cohort created successfully.\n")
cat("Total ICU admissions:", nrow(cohort), "\n")
cat("Unique patients:", n_distinct(cohort$subject_id), "\n")
cat("Age range:", min(cohort$age, na.rm = TRUE), "-", max(cohort$age, na.rm = TRUE), "years\n")
cat("Female proportion:", round(mean(cohort$gender == "F") * 100, 1), "%\n")

# Guardar cohort per a ús pels companys (opcional però recomanat)
# Assegura't que la carpeta 'data/' existeix
dir.create("data", showWarnings = FALSE)
write.csv(cohort, "data/cohort.csv", row.names = FALSE)


Cohort created successfully.
Total ICU admissions: 53424 
Unique patients: 38598 
Age range: 16 - 311 years
Female proportion: 43.7 %


# PART 2: Sepsis prevalence

In [4]:
# Define sepsis using ICD-9-CM codes
sepsis_adm <- tbl(con, "DIAGNOSES_ICD") %>%
  filter(
    substr(ICD9_CODE, 1, 3) == "038" |
    ICD9_CODE %in% c("99591", "99592", "78552")
  ) %>%
  select(HADM_ID) %>%
  distinct() %>%
  collect()
# Standardize column names to lowercase for consistency
colnames(sepsis_adm) <- tolower(colnames(sepsis_adm))


In [5]:
cohort_sepsis <- cohort %>%
  mutate(
    sepsis = if_else(hadm_id %in% sepsis_adm$hadm_id, 1, 0)  # 1 = with sepsis, 0 = without sepsis
  )

In [6]:
# Calculate the total volume and proportion (prevalence) of sepsis cases within the ICU
sepsis_summary <- cohort_sepsis %>%
  summarise(
    total_icu_admissions = n(),
    sepsis_icu_admissions = sum(sepsis),
    sepsis_prevalence = sepsis_icu_admissions / total_icu_admissions
  )

sepsis_summary


total_icu_admissions,sepsis_icu_admissions,sepsis_prevalence
<int>,<dbl>,<dbl>
53424,7261,0.1359127


In [7]:
table(cohort_sepsis$sepsis)


    0     1 
46163  7261 